In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080') #크롬드라이버 창크기
options.add_argument("headless") #창 숨기기
options.add_argument('disable-gpu') #그래픽 성능 낮춰서 크롤링 성능 높아기
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome(options=options)

In [ ]:
df = []
load = 1

url = 'https://www.teacherville.co.kr/trainguide/review/reviewList.edu'
driver.get(url)
time.sleep(5)

# 스크롤 내리기(첫 페이지에서)
driver.implicitly_wait(10)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 다음 페이지로(첫 페이지에서)
wait = WebDriverWait(driver, 30)
wait.until(EC.element_to_be_clickable((By.XPATH, '''//*[@id="divPaging"]/a[10]''')))
driver.find_element(by='xpath', value=f'''//*[@id="divPaging"]/a[10]''').click()


for _ in range(100):
    
    time.sleep(3)
    
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    # 다음 페이지로
    wait.until(EC.element_to_be_clickable((By.XPATH, '''//*[@id="divPaging"]/a[12]''')))
    driver.find_element(by='xpath', value= f'''//*[@id="divPaging"]/a[12]''').click()

In [ ]:
while load:

    li=[3,4,5,6,7,8,9,10,11,12]
        
    for num in li:
        
        # 전체 페이지 내용 불러오기
        hs = driver.page_source
        soup = BeautifulSoup( hs, 'html.parser')
        h = soup.find('ul',{'id':'reviewResultList'})
        
        # 2021년 10월부터 작성된 후기만 출력
        # 2021년 9월에 작성된 데이터가 나오면 해당 페이지까지만 출력하고 반복문 멈춤
        if date.split('/')[:2] == ['2021', '09']:
            load = 0
            break
            
        # 후기 내용을 특성별로 불러오기
        for i in h.find_all('li'):
            date = i.find('span','day').text
            hour = i.find('strong','credit').text
            name = i.find_all('p')[1].text.split(']')[1].strip()
            cate = i.find_all('p')[1].text.split(']')[0].split('/')[1].strip()
            text = i.find('p','text on').text.strip()
            star = len(i.find_all('span','star on'))
            df.append([hour,name,text,star,cate,date])


        # 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # 다음 페이지로
        wait.until(EC.element_to_be_clickable((By.XPATH, f'''//*[@id="divPaging"]/a[{num}]''')))
        driver.find_element(by='xpath', value= f'''//*[@id="divPaging"]/a[{num}]''').click()
        time.sleep(3)

In [ ]:
data = pd.DataFrame(df, columns = ['credit', 'subject', 'content', 'rating', 'category', 'date'])
data.to_csv('티처빌.csv', encoding = 'utf-8-sig')